<div>
<b><br><font face = "Gill Sans" size = "4"><center>Heart attack prediction with logistic regression</center></font></b>
</div>

In [82]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [83]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Check this site for the latest download link
# https://www.apache.org/dyn/closer.lua/spark
!wget -q https://dlcdn.apache.org/spark/spark-3.5.2/spark-3.5.2-bin-hadoop3.tgz
!tar xf spark-3.5.2-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
51 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InR

<span style="font-family:Gill Sans">
<h1>Hearth attach prediction with logisic regression</h1></span>

Let us use PySpark to predict if a person can suffer a heart attack based on some medical information.

Download the dataset from [Kaggle](https://www.kaggle.com/datasets/dileep070/heart-disease-prediction-using-logistic-regression?resource=download)

In [84]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, FloatType, IntegerType

# Create Spark Session in localhost
spark = SparkSession.builder.master("local").\
    appName("ITESO-LogisticRegression").\
    getOrCreate()

# Set spark context
sc = spark.sparkContext
sc.setLogLevel("ERROR")

heart_prediction_schema = StructType([
    StructField("male", IntegerType(), True),
    StructField("age", IntegerType(), True),
    StructField("education", IntegerType(), True),
    StructField("currentSmoker", IntegerType(), True),
    StructField("cigsPerDay", IntegerType(), True),
    StructField("BPMeds", IntegerType(), True),
    StructField("prevalentStroke", IntegerType(), True),
    StructField("prevalentHyp", IntegerType(), True),
    StructField("diabetes", IntegerType(), True),
    StructField("totChol", FloatType(), True),
    StructField("sysBP", FloatType(), True),
    StructField("diaBP", FloatType(), True),
    StructField("BMI", FloatType(), True),
    StructField("heartRate", IntegerType(), True),
    StructField("glucose", IntegerType(), True),
    StructField("TenYearCHD", IntegerType(), True)
])

heart_df = spark.read.format("csv").\
    option("header", "true").\
    option("mode", "permissive").\
    option("path", "/content/drive/MyDrive/Colab Notebooks/datasets/heart_attack_prediction.csv").\
    schema(heart_prediction_schema).\
    load()
heart_df.printSchema()
heart_df.show(n=10, truncate=False)

root
 |-- male: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- education: integer (nullable = true)
 |-- currentSmoker: integer (nullable = true)
 |-- cigsPerDay: integer (nullable = true)
 |-- BPMeds: integer (nullable = true)
 |-- prevalentStroke: integer (nullable = true)
 |-- prevalentHyp: integer (nullable = true)
 |-- diabetes: integer (nullable = true)
 |-- totChol: float (nullable = true)
 |-- sysBP: float (nullable = true)
 |-- diaBP: float (nullable = true)
 |-- BMI: float (nullable = true)
 |-- heartRate: integer (nullable = true)
 |-- glucose: integer (nullable = true)
 |-- TenYearCHD: integer (nullable = true)

+----+---+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+-----+---------+-------+----------+
|male|age|education|currentSmoker|cigsPerDay|BPMeds|prevalentStroke|prevalentHyp|diabetes|totChol|sysBP|diaBP|BMI  |heartRate|glucose|TenYearCHD|
+----+---+---------+-------------+----------+------+----

In [85]:
# Find Count of Null, None, NaN of All DataFrame Columns
from pyspark.sql.functions import col,isnan, when, count
heart_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in heart_df.columns]).show()

+----+---+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+---+---------+-------+----------+
|male|age|education|currentSmoker|cigsPerDay|BPMeds|prevalentStroke|prevalentHyp|diabetes|totChol|sysBP|diaBP|BMI|heartRate|glucose|TenYearCHD|
+----+---+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+---+---------+-------+----------+
|   0|  0|      105|            0|        29|    53|              0|           0|       0|     50|    0|    0| 19|        1|    388|         0|
+----+---+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+---+---------+-------+----------+



In [86]:
# Remove rows with at least one value as null
heart_df_cleaned = heart_df.na.drop()

In [87]:
heart_df_cleaned.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in heart_df.columns]).show()

+----+---+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+---+---------+-------+----------+
|male|age|education|currentSmoker|cigsPerDay|BPMeds|prevalentStroke|prevalentHyp|diabetes|totChol|sysBP|diaBP|BMI|heartRate|glucose|TenYearCHD|
+----+---+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+---+---------+-------+----------+
|   0|  0|        0|            0|         0|     0|              0|           0|       0|      0|    0|    0|  0|        0|      0|         0|
+----+---+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+---+---------+-------+----------+



In [90]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression

# Assemble the features into a single vector column
assembler = VectorAssembler(
    inputCols=["male", "age", "education", "currentSmoker", "cigsPerDay", "BPMeds",
               "prevalentStroke", "prevalentHyp", "diabetes", "totChol", "sysBP",
               "diaBP", "BMI", "heartRate", "glucose"],
    outputCol="features"
)
heart_assembler = assembler.transform(heart_df_cleaned)

# Split the data into training and test sets 70% training data and 30% testing data
train_data, test_data = heart_assembler.randomSplit([0.7, 0.3], seed=1523)

# Create a logistic regression model
lr = LogisticRegression(featuresCol="features", labelCol="TenYearCHD")

# Train to get the model
lr_model = lr.fit(train_data)|

In [91]:
# Print coefficients
print("Coefficients:", lr_model.coefficients)
print("Intercept:", lr_model.intercept)

# Make predictions on the test set
predictions = lr_model.transform(test_data)
predictions.select("male", "age", "TenYearCHD", "prediction").show(10, truncate=False)

Coefficients: [0.504095028009014,0.06973584250980697,-0.06854467210732668,0.1440297563481852,0.018266901884808968,0.26436138867904135,1.1299423733530594,0.056304961838082056,-0.26610928228148445,0.0023100403305663284,0.019129687346781922,-0.006964750918068415,-0.008002924247673434,-0.0027494904000263214,0.00560051579639078]
Intercept: -8.395063016948823
+----+---+----------+----------+
|male|age|TenYearCHD|prediction|
+----+---+----------+----------+
|0   |32 |0         |0.0       |
|0   |34 |0         |0.0       |
|0   |34 |0         |0.0       |
|0   |34 |0         |0.0       |
|0   |35 |0         |0.0       |
|0   |35 |0         |0.0       |
|0   |35 |0         |0.0       |
|0   |35 |0         |0.0       |
|0   |35 |0         |0.0       |
|0   |35 |1         |0.0       |
+----+---+----------+----------+
only showing top 10 rows

